# Example 2

The previous example needed a few adjustments, some of which are described in more detail below: 

- values of the slider limits were changed
- the `az` parameter was fixed (i.e., put inside the function)
- adding calculated value of $R_d$ to the plot
- adding a profile view of concentration

## Add $R_d$ to Plot

It may be useful to see what the calculated value of $R_d$ is. This is easy to add with the `text` method from Matplotlib, but I always have trouble remembering the name of the method as well as the arguments that are allowed. This is an excellent thing to ask an AI assistant, for example, the following question:

>Add a text description to the plot that shows the value of R.

Resulted in this line of code:

```python
plt.text(0.05, 0.10, f'R = {R:.2f}', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')
```

## Add profile view

This example also extends the previous code by creating a "profile view" of the concentration along the x-axis, which is also included in the educational materials (but not the original Matlab code). First, a few questions were asked to Copilot to ask it to restructure the functions and create a profile plot; the result had to be adjusted slightly (to get $R_d$ to be passed out of the concentration function).

## Code for Example 2

The resulting code is provided here.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

def get_concentration(M, x, y, ax, ay, Kp, lambda_):
    De = 0.6e-4  # molecular diffusion coefficient [m^2/d]
    az = 0.003
    n = 0.33  # porosity
    vx = 0.081  # seepage velocity, [m/d]
    rd = 1.810  # bulk density of the soil [g/cm^3]
    R = 1 + ((rd * Kp) / n)  # Retardation factor
    vrx = vx / R
    Dx = (ax * vx + De) / R
    Dy = (ay * vx + De) / R
    Dz = (az * vx + De) / R
    z = 0.0  # depth [m]
    t = 160  # time [d]
    return ((M / (n * 8 * ((np.pi * t) ** (3 / 2)) * ((Dx * Dy * Dz) ** 0.5))) * \
           np.exp((-(x - vrx * t) ** 2 / (4 * Dx * t)) - (y ** 2 / (4 * Dy * t)) - \
                  (z ** 2 / (4 * Dz * t)) - (lambda_ * t)), R)

def plot_contours(M, ax, ay, Kp, lambda_):
    xmin, xmax = -20, 30
    ymin, ymax = -20, 30
    x, y = np.meshgrid(np.linspace(xmin, xmax, 200), np.linspace(ymin, ymax, 200))
    c = np.zeros((200, 200))
    for i in range(200):
        for j in range(200):
            c[j, i], R = get_concentration(M, x[0, i], y[j, 0], ax, ay, Kp, lambda_)
    c = 1000 * c  # convert to [g/l]
    plt.figure()
    plt.contour(x, y, c, levels=20)
    plt.text(0.05, 0.10, f'R = {R:.2f}', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')
    plt.xlabel('X (m)')
    plt.ylabel('Y (m)')
    plt.title('Concentration, c [g/l]')
    plt.colorbar(label='Concentration [g/l]')
    plt.grid(True)
    plt.show()

def plot_profile(M, ax, ay, Kp, lambda_, y_val=0):
    xmin, xmax = -20, 30
    x = np.linspace(xmin, xmax, 200)
    c = np.zeros(200)
    for i in range(200):
        c[i], _ = get_concentration(M, x[i], y_val, ax, ay, Kp, lambda_)
    c = 1000 * c  # convert to [g/l]
    plt.figure()
    plt.plot(x, c)
    plt.xlim([-20, 30])
    plt.ylim([0, 100])
    plt.xlabel('X (m)')
    plt.ylabel('Concentration [g/l]')
    plt.title(f'Concentration Profile at Y = {y_val:.1f} m')
    plt.grid(True)
    plt.show()

M_slider = widgets.FloatSlider(value=0.370, min=0.1, max=1, step=0.1, description='M [g]:')
ax_slider = widgets.FloatSlider(value=0.300, min=0.1, max=10, step=0.1, description='ax [m]:')
ay_slider = widgets.FloatSlider(value=0.060, min=0.1, max=10, step=0.1, description='ay [m]:')
Kp_slider = widgets.FloatSlider(value=0.164, min=0.1, max=10, step=0.1, description='Kp [cm^3/g]:')
lambda_slider = widgets.FloatSlider(value=0.004, min=0.001, max=0.01, step=0.001, description='lambda [d^-1]:')

interact(plot_contours, M=M_slider, ax=ax_slider, ay=ay_slider, Kp=Kp_slider, lambda_=lambda_slider);
interact(plot_profile, M=M_slider, ax=ax_slider, ay=ay_slider, Kp=Kp_slider, lambda_=lambda_slider,
         y_val=widgets.FloatSlider(value=0, min=-20, max=20, step=0.1, description='Y [m]:'));

interactive(children=(FloatSlider(value=0.37, description='M [g]:', max=1.0, min=0.1), FloatSlider(value=0.3, …

interactive(children=(FloatSlider(value=0.37, description='M [g]:', max=1.0, min=0.1), FloatSlider(value=0.3, …